# S3 Access Dual-Pol Examples

This notebook uses `nisarh5toimage` to extract data from various images stored in s3 buckets. For this notebook, a single band of dual-pol image is extracted. 

In [1]:
!nisarh5toimage --help

usage: nisarh5toimage [-h] [--info] [--quickLook] [--ros3] [--conserveMem]
                      [--productFamily {RSLC,ROFF,RIFG,RUNW,GCOV,GUNW,GOFF}]
                      [--frequencyB] [--dB] [--sigma0] [--scaleToPixels]
                      [--wrapped] [--polarization {HH,VV,HV,VH}]
                      [--downsampleFactor DOWNSAMPLEFACTOR]
                      [--fields FIELDS [FIELDS ...]]
                      [--layers {1,2,3} [{1,2,3} ...]]
                      [--outputFormat {COG,GTiff,binary}]
                      productName [output]

Convert NISAR H5 product to Tiff(s), COG(s) or binary files

positional arguments:
  productName           NISAR h5 product (file path, s3, or https)
  output                Root name for ouput. Band (e.g., .unwrappedPhase) and
                        file type (e.g., .tiff) will be appended. (Not
                        required for --info)

options:
  -h, --help            show this help message and exit
  --info                Print 

## File Locations

In [2]:
tmpOut = '/home/jovyan/tmpRSLC-DP'

In [3]:
DHFile = 's3://nisar-adt-batch/commissioning/adt-on-demand/products/L1_L_RSLC/2025/10/05/NISAR_L1_PR_RSLC_006_107_D_004_4005_DHDH_A_20251005T182928_20251005T182943_A00408_P_P_J_001/NISAR_L1_PR_RSLC_006_107_D_004_4005_DHDH_A_20251005T182928_20251005T182943_A00408_P_P_J_001.h5'

In [4]:
#!aws s3 ls {DHFile}

## Case 1: Header Only

This command just reads the header info and prints a quick summary. By default it uses `ros3`.

In [5]:
!nisarh5toimage {DHFile} --info

page_buf_size 2147.483648 MB
Opening s3://nisar-adt-batch/commissioning/adt-on-demand/products/L1_L_RSLC/2025/10/05/NISAR_L1_PR_RSLC_006_107_D_004_4005_DHDH_A_20251005T182928_20251005T182943_A00408_P_P_J_001/NISAR_L1_PR_RSLC_006_107_D_004_4005_DHDH_A_20251005T182928_20251005T182943_A00408_P_P_J_001.h5 with ros3
loading as power False
Data loaded

Default polarization:
HH HV 
Available fields: 
HH
HV

Select Parameters
RSLC.track = 107 
RSLC.referenceOrbit = 972 
RSLC.frame = 4 
RSLC.datetime = 2025-10-05 18:29:28 
RSLC.centerLat = 5.695506953569062 
RSLC.centerLon = -0.9665153558294283 
RSLC.referenceGranule = None
RSLC.rangeBandwidth = 40000000.0 
RSLC.SLCNearRange = 882039.3768456667 
RSLC.SLCFarRange = 1047371.7945945625 
RSLC.SLCFirstZeroDopplerTime = 66568.0 
RSLC.SLCLastZeroDopplerTime = 66583.99934210526 
RSLC.LookDirection = left 
RSLC.PassType = descending 
RSLC.Wavelength = 0.2419632429378531 
RSLC.PRF = 1520.0 
RSLC.epsg = 4326 
RSLC.NumberRangeLooks = 1 
RSLC.NumberAzimuthL

## Case 2 Read DH band using Memory and Ros3

For this case, there are four possible ways to read the hdf5 file:
1) Stream file to memory and convert the data to numpy arrays,
2) Use `ros3` and convert to numpy arrays,
3) Stream file to memory, and write the data in blocks so no full numpy array is generated, and
4) Use `ros3` and block writes.

This example only reads half the data in the file (HH band of dual pol product), which can reduce the network transfer with the `ros3` driver.

### 1) Stream to Memory, Convert to Numpy

By default, the files is streamed to memory (see [codeblock](https://github.com/fastice/nisarhdf/blob/57648610195d7b345ebe2779f4e7552d3d8ce332/nisarhdf/nisarBaseHDF.py#L506)). This approach can be fast, but generate additional network traffic can occur if reading only a single band from a multi-band image.

In [6]:
!nisarh5toimage {DHFile} {tmpOut} --outputFormat binary --fields HH

[INFO] Downloading s3://nisar-adt-batch/commissioning/adt-on-demand/products/L1_L_RSLC/2025/10/05/NISAR_L1_PR_RSLC_006_107_D_004_4005_DHDH_A_20251005T182928_20251005T182943_A00408_P_P_J_001/NISAR_L1_PR_RSLC_006_107_D_004_4005_DHDH_A_20251005T182928_20251005T182943_A00408_P_P_J_001.h5 into memory...
[INFO] File downloaded into memory, opening with h5py...
loading as power False
Data loaded

Run time: 
Load data: 0:01:23.452023
Write data: 0:00:03.693002
Total 0:01:27.145025

Network traffic: 
Bytes sent:   27.6M
Bytes received:   10.9G

Memory Usage: 
Current memory usage:   25.2M
Peak memory usage:   30.5G


### 2) Use Ros 3, Convert to Numpy

Here is the same command again only using `ros3` (see [codeblock](https://github.com/fastice/nisarhdf/blob/57648610195d7b345ebe2779f4e7552d3d8ce332/nisarhdf/nisarBaseHDF.py#L546)). With `ros3` the process is a bit slower, but in this case the network transfer is halved and less memory is required.

In [7]:
!nisarh5toimage {DHFile} {tmpOut} --outputFormat binary --ros3 --fields HH

page_buf_size 2147.483648 MB
Opening s3://nisar-adt-batch/commissioning/adt-on-demand/products/L1_L_RSLC/2025/10/05/NISAR_L1_PR_RSLC_006_107_D_004_4005_DHDH_A_20251005T182928_20251005T182943_A00408_P_P_J_001/NISAR_L1_PR_RSLC_006_107_D_004_4005_DHDH_A_20251005T182928_20251005T182943_A00408_P_P_J_001.h5 with ros3
loading as power False
Data loaded

Run time: 
Load data: 0:02:02.488979
Write data: 0:00:03.769479
Total 0:02:06.258458

Network traffic: 
Bytes sent:   23.7M
Bytes received:    4.9G

Memory Usage: 
Current memory usage:    8.5M
Peak memory usage:   19.2G


### 3) Stream to Memory, No Numpy (--conserveMem)

This still reads the whole file, but is more memory efficient by using [block writes](https://github.com/fastice/nisarhdf/blob/99cef9c089a0a7b5b74b8864a3761efcfa31d91d/nisarhdf/nisarBaseHDF.py#L1740). In this exampe, it is the faster than the first method by about 1s.

In [8]:
!nisarh5toimage {DHFile} {tmpOut} --outputFormat binary --conserveMem --fields HH

[INFO] Downloading s3://nisar-adt-batch/commissioning/adt-on-demand/products/L1_L_RSLC/2025/10/05/NISAR_L1_PR_RSLC_006_107_D_004_4005_DHDH_A_20251005T182928_20251005T182943_A00408_P_P_J_001/NISAR_L1_PR_RSLC_006_107_D_004_4005_DHDH_A_20251005T182928_20251005T182943_A00408_P_P_J_001.h5 into memory...
[INFO] File downloaded into memory, opening with h5py...
loading as power False
Data loaded
Writing 24320 rows in blocks of 4608 rows (aligned to 512)...
Wrote rows 23040-24320 (100.0%)
Done.

Run time: 
Load data: 0:00:46.058742
Write data: 0:00:40.373822
Total 0:01:26.432564

Network traffic: 
Bytes sent:   34.4M
Bytes received:   10.9G

Memory Usage: 
Current memory usage:   25.2M
Peak memory usage:   15.1G


### 4) Ros 3, no Numpy (--conserveMem)

Like the other `ros3` case this is about 30s slower for this example and only downloads the data requested. Combined with the block writes,  this case uses by far the least memory and is definitely the preferred mode for a memory-limited computer. 

In [9]:
!nisarh5toimage {DHFile} {tmpOut} --outputFormat binary --ros3 --conserveMem --fields HH

page_buf_size 2147.483648 MB
Opening s3://nisar-adt-batch/commissioning/adt-on-demand/products/L1_L_RSLC/2025/10/05/NISAR_L1_PR_RSLC_006_107_D_004_4005_DHDH_A_20251005T182928_20251005T182943_A00408_P_P_J_001/NISAR_L1_PR_RSLC_006_107_D_004_4005_DHDH_A_20251005T182928_20251005T182943_A00408_P_P_J_001.h5 with ros3
loading as power False
Data loaded
Writing 24320 rows in blocks of 4608 rows (aligned to 512)...
Wrote rows 23040-24320 (100.0%)
Done.

Run time: 
Load data: 0:00:01.307471
Write data: 0:01:59.419554
Total 0:02:00.727025

Network traffic: 
Bytes sent:   23.7M
Bytes received:    4.9G

Memory Usage: 
Current memory usage:    8.5M
Peak memory usage:    3.7G
